In [39]:
import pandas as pd
from config import CONFIGS
import os
import re
import os
import json
import gc
from typing import Tuple

from utils.processing_functions import load_file_local_first, save_file_local_first, explode_columnar_df
from utils.weaviate_client import WeaviateClient

from sklearn.preprocessing import MinMaxScaler


ENVIRONMENT = os.environ.get("ENVIRONMENT", "dev")
S3_SCRAPER_BUCKET = CONFIGS["s3_scraper_bucket"]
GAME_CONFIGS = CONFIGS["games"]
RATINGS_CONFIGS = CONFIGS["ratings"]
IS_LOCAL = True if os.environ.get("IS_LOCAL", "False").lower() == "true" else False

read_dir = "data/prod/games/game_dfs_clean/"

weaviate_client = WeaviateClient()
weaviate_client.connect_weaviate_client_docker()


/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/weaviate/warnings.py:329: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


### Content Similarity

In [40]:
name_df = pd.read_pickle(f"{read_dir}games_clean.pkl")
name_df.columns

Index(['BGGId', 'Name', 'Description', 'ImagePath', 'NumAlternates',
       'NumExpansions', 'NumImplementations', 'IsReimplementation',
       'Rank:boardgame', 'BestPlayers', 'GoodPlayers', 'YearPublished',
       'MinPlayers', 'MaxPlayers', 'AvgRating', 'BayesAvgRating', 'StdDev',
       'NumOwned', 'NumWant', 'NumWish', 'NumWeightVotes', 'GameWeight',
       'MfgPlaytime', 'ComMinPlaytime', 'ComMaxPlaytime', 'MfgAgeRec',
       'NumUserRatings', 'ComAgeRec', 'LanguageEase', 'Family', 'Kickstarted',
       'Rank:strategygames', 'Rank:abstracts', 'Rank:partygames',
       'Rank:wargames', 'Rank:thematic', 'Rank:familygames',
       'Rank:childrensgames', 'Rank:cgs', 'Cat:Thematic', 'Cat:Strategy',
       'Cat:War', 'Cat:Family', 'Cat:CGS', 'Cat:Abstract', 'Cat:Party',
       'Cat:Childrens'],
      dtype='object')

In [41]:
mech_df = pd.read_pickle(f"{read_dir}mechanics_clean.pkl")
mech_df.head()

,BGGId,mechanic
0,1,Simultaneous Action Selection
1,1,Area Majority / Influence
2,1,Alliances
3,1,Negotiation
4,1,Dice Rolling


In [42]:
mech_df = explode_columnar_df(mech_df)
mech_df.head()

mechanic,Acting,Action / Dexterity,Action / Event,Action Points,Action Queue,Action Retrieval,Action Timer,Advantage Token,Alliances,Area Majority / Influence,...,Traitor Game,Trick-taking,Tug of War,Variable Phase Order,Variable Player Powers,Variable Set-up,Victory Points as a Resource,Voting,Worker Placement,Zone of Control
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [43]:
name_df = pd.read_pickle(f"{read_dir}games_clean.pkl")
name_df = name_df[['BGGId','Name']]
df = pd.read_pickle("top_1000_cleaned_rag_with_ratings_extrap.pkl")
df = name_df.merge(df, on="BGGId", how="inner")
df = df.drop(columns=["Description","Positive_Components","Negative_Components","Positive_Sentences","Negative_Sentences"])
df = df.rename(columns={x:x.lower().replace(" ","_") for x in df.columns.tolist()})
df = df.fillna(0)
small_df = df[['bggid',	'name',	'about']]
del name_df
gc.collect()

/Users/jennywadkins/.pyenv/versions/3.12.6/lib/python3.12/asyncio/selector_events.py:879: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=78 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


846

### Match Keywords Only

In [51]:
collection_name = "attributes"

In [7]:
keyword_df = df.copy().drop(columns=['about'])
keyword_df.head()

,bggid,name,positive_strategic_depth,positive_excellent_design,positive_player_interaction,positive_replayability,positive_thematic_integration,positive_component_quality,negative_steep_learning_curve,negative_fiddly_mechanics,...,negative_anticlimactic_endings,positive_excellent_production_values,positive_attractive_artwork,positive_accessible_non_gamers,positive_variable_experiences,negative_catch_up_mechanics,positive_simple_to_learning,negative_action_limitations,positive_good_themes,positive_good_interactions
0,1,Die Macher,1.000,0.847,0.907,0.945,0.718,0.810,0.871,0.819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100423,Elder Sign,0.824,0.817,0.749,0.916,0.707,0.915,0.791,0.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100901,Flash Point: Fire Rescue,0.860,0.795,0.883,0.926,0.000,0.850,0.810,0.965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101721,Mage Wars Arena,1.000,0.847,0.792,1.000,0.718,0.803,0.931,0.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,102652,Sentinels of the Multiverse,0.805,0.820,0.907,1.000,0.759,0.767,0.849,0.812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
positive_columns = [x for x in keyword_df.columns if x.startswith('positive_')]
negative_columns = [x for x in keyword_df.columns if x.startswith('negative_')]

all_columns_to_produce = positive_columns + negative_columns

# scaler = MinMaxScaler(feature_range=(0,10))

# for col in all_columns_to_produce:
#     keyword_df[col] = scaler.fit_transform(keyword_df[[col]])

all_columns_to_produce[:5]

keyword_df.head()

,bggid,name,positive_strategic_depth,positive_excellent_design,positive_player_interaction,positive_replayability,positive_thematic_integration,positive_component_quality,negative_steep_learning_curve,negative_fiddly_mechanics,...,negative_anticlimactic_endings,positive_excellent_production_values,positive_attractive_artwork,positive_accessible_non_gamers,positive_variable_experiences,negative_catch_up_mechanics,positive_simple_to_learning,negative_action_limitations,positive_good_themes,positive_good_interactions
0,1,Die Macher,1.000,0.847,0.907,0.945,0.718,0.810,0.871,0.819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100423,Elder Sign,0.824,0.817,0.749,0.916,0.707,0.915,0.791,0.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100901,Flash Point: Fire Rescue,0.860,0.795,0.883,0.926,0.000,0.850,0.810,0.965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101721,Mage Wars Arena,1.000,0.847,0.792,1.000,0.718,0.803,0.931,0.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,102652,Sentinels of the Multiverse,0.805,0.820,0.907,1.000,0.759,0.767,0.849,0.812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=False, attributes=all_columns_to_produce)

keyword_df = weaviate_client.add_bgg_collection_batch(df=keyword_df, collection_name=collection_name, use_about=False, attributes=all_columns_to_produce)

Collection attributes already exists for this block
Deleted and recreating collection attributes


In [53]:
# uuid = keyword_df.loc[keyword_df['name']=='Pandemic Legacy: Season 1']['UUID'].values[0]
# uuid = keyword_df.loc[keyword_df['name']=='Gloomhaven']['UUID'].values[0]
uuid = keyword_df.loc[keyword_df['name']=='Great Western Trail']['UUID'].values[0]
similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=20)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(keyword_df, on='UUID', how='inner')[1:]
picks[['UUID','distance','bggid','name']].head(10)

,UUID,distance,bggid,name
1,b5b776e1-561f-51f1-bc76-3fa8fed8f21d,0.176939,341169,Great Western Trail: Second Edition
2,fe9311b6-fbcf-54a6-a80b-a936d1ec0816,0.276424,380607,Great Western Trail: New Zealand
3,72b56190-3940-5d40-80f9-cf1508a82941,0.276494,364011,Great Western Trail: Argentina
4,3a52b286-fddc-5322-88be-ea1fee31d9c0,0.499826,390092,Ticket to Ride Legacy: Legends of the West
5,a95ecbbe-d6cb-5412-b035-cf14827f63d2,0.514818,281655,High Frontier 4 All
6,d60b86a8-dc56-534a-ab18-30f6c43e5cd1,0.528814,232405,Western Legends
7,49980f64-4d24-5333-88d0-96750c30010c,0.531522,283864,Trails of Tucana
8,8ad7571a-8a31-5cb0-98f0-db214d82fb89,0.542588,379078,Expeditions
9,df2f648b-4d07-5817-b43e-100bcfd6bd19,0.546637,255692,New Frontiers
10,298e2ab9-17bf-5ea6-acee-fa85d6d51f9c,0.559694,329591,Ultimate Railroads


### Without attributes

In [26]:
collection_name = "similarities"

small_df.head()

,bggid,name,about,UUID
0,1,Die Macher,This game revolves around political strategy a...,4952cb7e-1504-56c0-99dc-e65311d96e27
1,100423,Elder Sign,This game is a cooperative board game that imm...,55ebfe86-0bac-5285-9691-cefa121b348e
2,100901,Flash Point: Fire Rescue,This game is a cooperative board game where pl...,046ea6fc-0372-542a-ac0f-a167f88e28d8
3,101721,Mage Wars Arena,This game is a tactical strategy experience th...,f62149b6-059f-5d38-a233-00398b49c12c
4,102652,Sentinels of the Multiverse,This game is a cooperative card game where pla...,9fd43008-1dd5-555d-b188-b4ec2509228c


In [12]:
weaviate_client.create_bgg_collection(collection_name=collection_name, reset=False, use_about=True)

small_df = weaviate_client.add_bgg_collection_batch(df=small_df, collection_name=collection_name, use_about=True)

/Users/jennywadkins/.local/share/virtualenvs/boardgamegeek-ZH0FNRKg/lib/python3.12/site-packages/weaviate/warnings.py:329: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/Users/jennywadkins/.pyenv/versions/3.12.6/lib/python3.12/asyncio/selector_events.py:879: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=86 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)


Collection similarities already exists for this block


/Users/jennywadkins/github_repos/boardgamegeek/utils/weaviate_client.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["UUID"] = uuids


In [28]:
# uuid = small_df.loc[small_df['name']=='Pandemic Legacy: Season 1']['UUID'].values[0]
uuid = small_df.loc[small_df['name']=='Gloomhaven']['UUID'].values[0]
# uuid = small_df.loc[small_df['name']=='Great Western Trail']['UUID'].values[0]
similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=20)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(small_df, on='UUID', how='inner')[1:]
picks.head(10)

,UUID,distance,bggid,name,about
1,eb626a2c-3a7a-5c99-9133-f12e6a3a48d7,0.159391,295770,Frosthaven,This game is a cooperative dungeon crawler tha...
2,d2936b9f-ea3b-5aba-8159-c57599773ce2,0.178758,291457,Gloomhaven: Jaws of the Lion,This game is a cooperative dungeon crawler tha...
3,6ccb1644-999b-5bd9-91ca-74d7ac689e2d,0.190315,286063,The 7th Citadel,This game is a narrative-driven adventure that...
4,4a23720a-c314-50f3-8a8e-a331ef6990f2,0.195274,150997,Shadows of Brimstone: Swamps of Death,This game is a cooperative dungeon crawler set...
5,f1578e22-cc52-57d5-8299-a82708b05ab5,0.196859,215341,Thunderstone Quest,This game is a blend of dungeon crawling and d...
6,48bf4cc2-48b4-5934-adde-cddac1759462,0.197532,322708,Descent: Legends of the Dark,This game is a cooperative dungeon crawler tha...
7,3273801d-9289-5e54-88e0-163a77d2a1cf,0.205126,181521,Warhammer Quest: The Adventure Card Game,This game is a cooperative dungeon crawler tha...
8,286c206f-d5e5-5972-b5d2-7b0d8fcdc137,0.209575,248562,Mage Knight: Ultimate Edition,"This game is a complex, strategic adventure th..."
9,622bef0c-be56-5f5d-98b3-0cff92119a13,0.209914,146791,Shadows of Brimstone: City of the Ancients,This game is a cooperative dungeon crawler tha...
10,e04f4662-eaf4-536a-aa78-bc695cb8b19b,0.210240,169427,Middara: Unintentional Malum – Act 1,This game is a narrative-driven dungeon crawle...


### With all attributes

In [44]:
collection_name = "all_attributes"
df.head()

,bggid,name,about,positive_strategic_depth,positive_excellent_design,positive_player_interaction,positive_replayability,positive_thematic_integration,positive_component_quality,negative_steep_learning_curve,...,negative_anticlimactic_endings,positive_excellent_production_values,positive_attractive_artwork,positive_accessible_non_gamers,positive_variable_experiences,negative_catch_up_mechanics,positive_simple_to_learning,negative_action_limitations,positive_good_themes,positive_good_interactions
0,1,Die Macher,This game revolves around political strategy a...,1.000,0.847,0.907,0.945,0.718,0.810,0.871,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100423,Elder Sign,This game is a cooperative board game that imm...,0.824,0.817,0.749,0.916,0.707,0.915,0.791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100901,Flash Point: Fire Rescue,This game is a cooperative board game where pl...,0.860,0.795,0.883,0.926,0.000,0.850,0.810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101721,Mage Wars Arena,This game is a tactical strategy experience th...,1.000,0.847,0.792,1.000,0.718,0.803,0.931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,102652,Sentinels of the Multiverse,This game is a cooperative card game where pla...,0.805,0.820,0.907,1.000,0.759,0.767,0.849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
positive_columns = [x for x in df.columns if x.startswith('positive_')]
negative_columns = [x for x in df.columns if x.startswith('negative_')]

all_columns_to_produce = positive_columns + negative_columns

# scaler = MinMaxScaler(feature_range=(0,.5))

# for col in all_columns_to_produce:
#     df[col] = scaler.fit_transform(df[[col]])

all_columns_to_produce[:5]

['positive_strategic_depth',
 'positive_excellent_design',
 'positive_player_interaction',
 'positive_replayability',
 'positive_thematic_integration']

In [46]:
weaviate_client.create_bgg_collection(collection_name=collection_name, reset=True, use_about=True, attributes=all_columns_to_produce)

df = weaviate_client.add_bgg_collection_batch(df=df, collection_name=collection_name, use_about=True, attributes=all_columns_to_produce)

Collection all_attributes already exists for this block
Deleted and recreating collection all_attributes


In [50]:
uuid = df.loc[df['name']=='Pandemic Legacy: Season 1']['UUID'].values[0]
# uuid = df.loc[df['name']=='Gloomhaven']['UUID'].values[0]
similars = weaviate_client.find_near_objects(collection_name=collection_name, uuid=uuid, limit=20)

picks = {}

for item in similars:
    picks[str(item.uuid)]=item.metadata.distance
    
picks = pd.DataFrame.from_dict(picks, columns=['distance'], orient='index').sort_values(by='distance', ascending=True).reset_index().rename(columns={'index':'UUID'})

picks = picks.merge(df, on='UUID', how='inner')
picks[['UUID','distance','bggid','name','about']].head(10)

,UUID,distance,bggid,name,about
0,671a3be9-785e-5d92-aa22-1a20ada02548,2.980232e-07,161936,Pandemic Legacy: Season 1,This game is a cooperative board game where pl...
1,f0306a8c-1e74-5f7c-8b17-ffde1a607c09,1.171774e-01,221107,Pandemic Legacy: Season 2,This game is a cooperative board game that emp...
2,e6b4d3fa-dd76-511e-adb3-239bee3f26c8,1.349763e-01,314040,Pandemic Legacy: Season 0,This game is a cooperative board game that com...
3,e1187bf0-765f-5e96-9ed6-a5326d61721d,1.612439e-01,30549,Pandemic,This game is a cooperative board game where pl...
4,6a455438-6781-581f-9876-69848e3bb292,2.247700e-01,150658,Pandemic: The Cure,This game is a cooperative board game where pl...
5,9e9c78f8-233a-5862-ba0f-9ef526a6529d,2.536179e-01,257499,Arkham Horror (Third Edition),This game is a cooperative board game set in a...
6,18c4d185-8ee4-5e74-8be0-03018a697ff1,2.563391e-01,240196,Betrayal Legacy,This game is a legacy-style board game that co...
7,65127133-696f-5e37-b2a4-40f759f1a0ac,2.634566e-01,150376,Dead of Winter: A Crossroads Game,This game is a cooperative survival experience...
8,1ba0bb29-bd44-5ae5-9b11-da69c40b2970,2.728389e-01,338960,Slay the Spire: The Board Game,This game is a cooperative deck-building exper...
9,b41b3026-9b3e-5254-aa1b-3ccca64d65da,2.741372e-01,169427,Middara: Unintentional Malum – Act 1,This game is a narrative-driven dungeon crawle...
